<a href="https://colab.research.google.com/github/thiagoigfraga/pesquisa_sisrec_interacoes/blob/main/top_k_artigos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install river

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 47.1 MB/s eta 0:00:00


In [ ]:
# Imports
import pandas as pd
import numpy as np
from river import forest, metrics, drift, stats
import random
import hashlib
import pickle
from datetime import datetime
from collections import Counter, defaultdict
import os
import traceback
from typing import Optional, Dict, Any

# Definir sementes para reprodutibilidade
random.seed(42)
np.random.seed(42)


def convert_unix_timestamp(ts):
    """Converte timestamp Unix em milissegundos para datetime"""
    return pd.to_datetime(ts, unit="ms")


def consistent_hash(value):
    """Função de hash consistente usando SHA-256"""
    return (
        int(hashlib.sha256(str(value).encode("utf-8")).hexdigest(), 16) % 1000
    )


def analyze_temporal_distribution(dataset):
    """
    Analisa a distribuição temporal dos dados
    """
    print("\n=== Análise Temporal ===")

    # Converte timestamps
    dataset["hour"] = dataset["timestamp"].dt.hour
    dataset["day"] = dataset["timestamp"].dt.date

    # Análise por hora
    hourly_dist = dataset.groupby("hour").size()
    print("\nDistribuição por hora do dia:")
    print(hourly_dist)

    # Análise por dia
    daily_dist = dataset.groupby("day").size()
    print("\nDistribuição por dia:")
    print(daily_dist)

    # Calcula intervalos entre interações
    dataset = dataset.sort_values("timestamp")
    dataset["time_diff"] = dataset["timestamp"].diff().dt.total_seconds()

    print("\nIntervalos entre interações (segundos):")
    print(dataset["time_diff"].describe())

    return hourly_dist, daily_dist


class OnlineNewsRecommender:
    def __init__(
        self,
        n_models=15,  # Aumentado de 10 para 15
        drift_detector=drift.ADWIN(delta=0.002),  # Ajustado delta
        top_k=10,
    ):
        # Mantém histórico temporal
        self.temporal_weights = {}  # Pesos temporais para artigos
        self.article_timestamps = defaultdict(
            list
        )  # Últimos timestamps por artigo
        self.time_window = 3600 * 6  # 6 horas em segundos

        # Inicializa um normalizador para cada feature numérica
        self.scalers = {
            "user_id_hash": stats.Mean(),
            "article_id_hash": stats.Mean(),
            "hour_sin": stats.Mean(),
            "hour_cos": stats.Mean(),
            "day_sin": stats.Mean(),
            "day_cos": stats.Mean(),
            "month_sin": stats.Mean(),
            "month_cos": stats.Mean(),
            "hour": stats.Mean(),
            "article_popularity": stats.Mean(),
            "user_activity": stats.Mean(),
            "temporal_weight": stats.Mean(),  # Adicionado
        }

        self.vars = {k: stats.Var() for k in self.scalers.keys()}

        # Modelo base com detector de drift
        self.model = forest.ARFClassifier(
            n_models=n_models,
            drift_detector=drift_detector,
            grace_period=50,
            max_features="sqrt",
            seed=42,
            leaf_prediction="nb",
        )

        # Métricas online
        self.metrics = {
            "accuracy": metrics.Accuracy(),
            "f1": metrics.F1(),
            "precision": metrics.Precision(),
            "recall": metrics.Recall(),
            "roc_auc": metrics.ROCAUC(),
            "log_loss": metrics.LogLoss(),
        }

        # Contadores e estado
        self.article_counter = Counter()
        self.user_counter = Counter()
        self.current_top = set()
        self.top_k = top_k

    def _calculate_temporal_weight(self, article_id, current_timestamp):
        """
        Calcula peso temporal do artigo baseado em suas interações recentes
        """
        recent_timestamps = [
            ts
            for ts in self.article_timestamps[article_id]
            if (current_timestamp - ts).total_seconds() <= self.time_window
        ]

        if not recent_timestamps:
            return 0.0

        # Peso decai exponencialmente com o tempo
        weights = np.exp(
            -0.1
            * np.array(
                [
                    (current_timestamp - ts).total_seconds()
                    / 3600  # Converte para horas
                    for ts in recent_timestamps
                ]
            )
        )

        return np.mean(weights)

    def _normalize_feature(self, name, value):
        """Normaliza uma feature usando média e variância online"""
        # Atualiza estatísticas
        self.scalers[name].update(value)
        self.vars[name].update(value)

        # Calcula z-score
        mean = self.scalers[name].get()
        var = self.vars[name].get()
        std = np.sqrt(var) if var > 0 else 1

        return (value - mean) / (std + 1e-8)

    def _extract_features(self, x):
        """
        Extrai as features do exemplo de entrada.
        """
        timestamp = convert_unix_timestamp(x["click_timestamp"])

        # Features básicas
        features = {
            "user_id_hash": consistent_hash(x["user_id"]),
            "article_id_hash": consistent_hash(x["click_article_id"]),
            "hour_sin": np.sin(2 * np.pi * timestamp.hour / 24),
            "hour_cos": np.cos(2 * np.pi * timestamp.hour / 24),
            "day_sin": np.sin(2 * np.pi * timestamp.dayofweek / 7),
            "day_cos": np.cos(2 * np.pi * timestamp.dayofweek / 7),
            "month_sin": np.sin(2 * np.pi * timestamp.month / 12),
            "month_cos": np.cos(2 * np.pi * timestamp.month / 12),
            "hour": timestamp.hour,
        }

        # Calcula popularidade excluindo a interação atual
        article_count = self.article_counter.get(x["click_article_id"], 0)
        user_count = self.user_counter.get(x["user_id"], 0)

        total_interactions = sum(self.article_counter.values()) or 1
        total_users = sum(self.user_counter.values()) or 1

        # Features temporais e de popularidade
        temporal_weight = self._calculate_temporal_weight(
            x["click_article_id"], timestamp
        )

        features.update(
            {
                "article_popularity": article_count / total_interactions,
                "user_activity": user_count / total_users,
                "temporal_weight": temporal_weight,
                "is_business_hour": int(9 <= timestamp.hour <= 18),
                "is_peak_hour": int(10 <= timestamp.hour <= 16),
                "day_of_week": timestamp.dayofweek,
                "is_weekend": int(timestamp.dayofweek >= 5),
            }
        )

        # Normaliza features numéricas
        normalized_features = {
            name: self._normalize_feature(name, value)
            for name, value in features.items()
            if name in self.scalers
        }

        # Adiciona features categóricas sem normalização
        normalized_features.update(
            {
                k: features[k]
                for k in [
                    "is_business_hour",
                    "is_peak_hour",
                    "day_of_week",
                    "is_weekend",
                ]
            }
        )

        # Atualiza histórico temporal
        self.article_timestamps[x["click_article_id"]].append(timestamp)
        if (
            len(self.article_timestamps[x["click_article_id"]]) > 1000
        ):  # Limita histórico
            self.article_timestamps[x["click_article_id"]] = (
                self.article_timestamps[x["click_article_id"]][-1000:]
            )

        return normalized_features

    def learn_one(self, x):
        """Aprende com um exemplo"""
        # Atualiza top articles antes de processar a interação atual para evitar vazamento
        if sum(self.article_counter.values()) % 100 == 0:
            self.current_top = set(
                article
                for article, _ in self.article_counter.most_common(self.top_k)
            )

        # Define target (1 se artigo está no top-k)
        target = int(x["click_article_id"] in self.current_top)

        # Extrai features
        features = self._extract_features(x)

        # Treina o modelo
        self.model.learn_one(features, target)

        # Atualiza contadores após predição
        self.article_counter[x["click_article_id"]] += 1
        self.user_counter[x["user_id"]] += 1

        # Atualiza métricas se houver predição
        pred = self.model.predict_one(features)
        if pred is not None:
            for metric in self.metrics.values():
                metric.update(target, pred)

        return self

    def predict_one(self, x):
        """Faz predição para um exemplo"""
        features = self._extract_features(x)
        return self.model.predict_one(features)

    def predict_proba_one(self, x):
        """Retorna probabilidades para um exemplo"""
        features = self._extract_features(x)
        return self.model.predict_proba_one(features)

    def save(self, path):
        """Salva o modelo completo"""
        with open(path, "wb") as f:
            pickle.dump(self, f)

    @classmethod
    def load(cls, path):
        """Carrega o modelo completo"""
        with open(path, "rb") as f:
            return pickle.load(f)


class NewsRecommenderSystem:
    """Sistema para gerenciar treinamento e predição do recomendador de notícias."""

    def __init__(
        self, model_path=None, max_samples=None, timestamp=None, config=None
    ):
        """
        Inicializa o sistema de recomendação.

        Parameters:
        -----------
        model_path : str, optional
            Caminho para o modelo existente.
        max_samples : int, optional
            Número máximo de amostras a serem processadas.
        timestamp : str, optional
            Timestamp para nomear arquivos.
        config : dict, optional
            Configurações do modelo.
        """
        self.model = None
        self.history = {
            "processed_records": 0,
            "start_time": datetime.now(),
            "metrics": {},
            "dataset_info": {},
            "model_info": {},
            "config": config or {},
        }
        self.max_samples = max_samples
        self.timestamp = timestamp or datetime.now().strftime("%Y%m%d_%H%M%S")

        if model_path:
            self._load_existing_model(model_path)
        else:
            self._initialize_new_model()

    def _load_existing_model(self, model_path):
        """Carrega modelo existente."""
        try:
            self.model = OnlineNewsRecommender.load(model_path)
            self.history["model_info"] = {
                "original_model_path": model_path,
                "load_time": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            }
        except Exception as e:
            raise Exception(f"Erro ao carregar modelo: {str(e)}")

    def _initialize_new_model(self):
        """Inicializa novo modelo."""
        n_models = self.history["config"].get("n_models", 15)
        delta = self.history["config"].get("delta", 0.002)  # Ajustado
        top_k = self.history["config"].get("top_k", 10)

        drift_detector = drift.ADWIN(delta=delta)

        self.model = OnlineNewsRecommender(
            n_models=n_models, drift_detector=drift_detector, top_k=top_k
        )

        # Salva as configurações do classificador
        self.history["model_info"] = {
            "initialization_time": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "model_type": "OnlineNewsRecommender",
            "n_models": n_models,
            "drift_detector": type(drift_detector).__name__,
            "drift_detector_params": drift_detector.__dict__,
            "top_k": top_k,
            "model_params": {
                "n_models": n_models,
                "grace_period": self.model.model.grace_period,
                "max_features": self.model.model.max_features,
                "seed": self.model.model.seed,
                "leaf_prediction": self.model.model.leaf_prediction,
            },
        }

    def validate_dataset(self, dataset: pd.DataFrame) -> bool:
        """
        Valida se o dataset tem as colunas necessárias.

        Parameters:
        -----------
        dataset : pd.DataFrame
            O dataset a ser validado.

        Returns:
        --------
        bool
            True se o dataset é válido, levanta ValueError caso contrário.
        """
        required_columns = ["user_id", "click_article_id", "click_timestamp"]
        missing_columns = [
            col for col in required_columns if col not in dataset.columns
        ]

        if missing_columns:
            raise ValueError(f"Colunas faltando no dataset: {missing_columns}")

        return True

    def _update_final_metrics(self):
        """
        Atualiza métricas finais no histórico com informações adicionais.
        """
        try:
            # Acessa drifts através do ARFClassifier
            n_drifts = sum(
                model.drift_detector.n_detections
                for model in self.model.model._models
                if hasattr(model, "drift_detector")
            )
            n_warnings = sum(
                model.drift_detector.n_warnings
                for model in self.model.model._models
                if hasattr(model, "drift_detector")
            )
            top_articles = dict(self.model.article_counter.most_common(10))

            self.history["metrics"] = {
                "final_accuracy": self.model.metrics["accuracy"].get(),
                "final_f1": self.model.metrics["f1"].get(),
                "final_precision": self.model.metrics["precision"].get(),
                "final_recall": self.model.metrics["recall"].get(),
                "roc_auc": self.model.metrics["roc_auc"].get(),
                "log_loss": self.model.metrics["log_loss"].get(),
                "processed_records": self.history["processed_records"],
                "processing_time": str(
                    datetime.now() - self.history["start_time"]
                ),
                "n_drifts": n_drifts,
                "n_warnings": n_warnings,
                "top_articles": str(top_articles),
            }
        except Exception as e:
            print(f"Erro atualizando métricas: {str(e)}")
            self.history["metrics"] = {
                "error": str(e),
                "processed_records": self.history["processed_records"],
                "processing_time": str(
                    datetime.now() - self.history["start_time"]
                ),
            }

    def analyze_samples(self, dataset):
        """
        Analisa e mostra informações sobre as amostras do dataset com validações adicionais.
        """
        try:
            print("\n=== Análise das Amostras ===")

            # Verifica duplicações
            duplicates = dataset.duplicated().sum()
            if duplicates > 0:
                print(f"Aviso: Encontradas {duplicates:,} linhas duplicadas")
                print("Removendo duplicatas...")
                dataset = dataset.drop_duplicates()

            # Contagem de registros
            total_samples = len(dataset)
            total_interactions = (
                dataset.groupby(
                    ["user_id", "click_article_id", "click_timestamp"]
                )
                .size()
                .sum()
            )

            if total_samples != total_interactions:
                print(f"Aviso: Possível inconsistência na contagem de amostras")
                print(f"Total de linhas: {total_samples:,}")
                print(f"Total de interações únicas: {total_interactions:,}")

            print(f"Total de amostras: {total_interactions:,}")
            print(
                f"Período: {dataset['timestamp'].min()} até {dataset['timestamp'].max()}"
            )
            print(f"Usuários únicos: {dataset['user_id'].nunique():,}")
            print(f"Artigos únicos: {dataset['click_article_id'].nunique():,}")

            # Verificar integridade dos dados
            print("\nVerificação de integridade:")
            null_counts = dataset.isnull().sum()
            if null_counts.any():
                print("Valores nulos encontrados:")
                print(null_counts[null_counts > 0])
                dataset = dataset.dropna()

            # Análise de timestamps
            invalid_timestamps = (
                pd.to_datetime(
                    dataset["click_timestamp"], unit="ms", errors="coerce"
                )
                .isnull()
                .sum()
            )
            if invalid_timestamps > 0:
                print(
                    f"\nAviso: {invalid_timestamps} timestamps inválidos encontrados"
                )
                dataset = dataset.dropna(subset=["click_timestamp"])

            # Análise temporal
            analyze_temporal_distribution(dataset)

            return dataset

        except Exception as e:
            print(f"Erro analisando amostras: {str(e)}")
            traceback.print_exc()
            return dataset

    def validate_temporal(self, dataset, validation_ratio=0.2):
        """
        Realiza validação temporal do modelo usando uma proporção dos dados ativos

        Parameters:
        -----------
        dataset : pd.DataFrame
            Dataset completo
        validation_ratio : float
            Proporção dos dados para validação (default: 0.2)
        """
        # Identifica dias ativos (>1000 interações)
        daily_counts = dataset.groupby(dataset["timestamp"].dt.date).size()
        active_days = daily_counts[daily_counts > 1000].index

        # Seleciona apenas os primeiros 16 dias (período consistente)
        active_days = sorted(active_days)[:16]

        # Filtra dataset para usar apenas dias ativos
        dataset = dataset[dataset["timestamp"].dt.date.isin(active_days)]

        # Ordena por timestamp
        dataset = dataset.sort_values("timestamp")

        # Calcula ponto de corte baseado na proporção
        n_samples = len(dataset)
        split_idx = int(n_samples * (1 - validation_ratio))
        split_time = dataset.iloc[split_idx]["timestamp"]

        # Divide dados
        train_data = dataset[dataset["timestamp"] <= split_time]
        val_data = dataset[dataset["timestamp"] > split_time]

        print(f"\nValidação Temporal:")
        print(
            f"Dados de treino: {len(train_data):,} registros "
            f"({(split_time - dataset['timestamp'].min()).days + 1} dias)"
        )
        print(
            f"Dados de validação: {len(val_data):,} registros "
            f"({(dataset['timestamp'].max() - split_time).days + 1} dias)"
        )

        # Treina com dados de treino
        print("\nTreinando modelo...")
        for _, row in train_data.iterrows():
            self.model.learn_one(row.to_dict())

        # Valida com dados de validação
        print("\nValidando modelo...")
        val_metrics = {
            "accuracy": metrics.Accuracy(),
            "f1": metrics.F1(),
            "precision": metrics.Precision(),
            "recall": metrics.Recall(),
            "roc_auc": metrics.ROCAUC(),
            "log_loss": metrics.LogLoss(),
        }

        predictions = []
        for _, row in val_data.iterrows():
            x = row.to_dict()
            pred = self.model.predict_one(x)
            proba = self.model.predict_proba_one(x)
            target = int(x["click_article_id"] in self.model.current_top)

            for name, metric in val_metrics.items():
                if name in ["roc_auc", "log_loss"]:
                    metric.update(target, proba)
                else:
                    metric.update(target, pred)

            predictions.append(
                {
                    "timestamp": x["timestamp"],
                    "user_id": x["user_id"],
                    "article_id": x["click_article_id"],
                    "prediction": pred,
                    "probability": proba.get(1, 0),
                    "target": target,
                }
            )

        print("\nMétricas de Validação:")
        for name, metric in val_metrics.items():
            print(f"{name}: {metric.get():.4f}")

        return val_metrics, predictions

    def save_history(self):
        """
        Salva o histórico do processamento em um arquivo CSV.
        """
        try:
            history_data = {
                "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "dataset_size": self.history["dataset_info"].get(
                    "total_records"
                ),
                "unique_users": self.history["dataset_info"].get(
                    "unique_users"
                ),
                "unique_articles": self.history["dataset_info"].get(
                    "unique_articles"
                ),
                "window_size": self.history["dataset_info"].get("window_size"),
                "step_size": self.history["dataset_info"].get("step_size"),
                "total_days": self.history["dataset_info"].get("total_days"),
                "processed_records": self.history["metrics"].get(
                    "processed_records"
                ),
                "accuracy": self.history["metrics"].get("final_accuracy"),
                "f1_score": self.history["metrics"].get("final_f1"),
                "precision": self.history["metrics"].get("final_precision"),
                "recall": self.history["metrics"].get("final_recall"),
                "roc_auc": self.history["metrics"].get("roc_auc"),
                "log_loss": self.history["metrics"].get("log_loss"),
                "processing_time": self.history["metrics"].get(
                    "processing_time"
                ),
                "n_drifts": self.history["metrics"].get("n_drifts"),
                "n_warnings": self.history["metrics"].get("n_warnings"),
                "top_articles": self.history["metrics"].get("top_articles"),
                "predictions_file": self.history.get("predictions_file"),
                "model_path": self.history["model_info"].get(
                    "final_model_path"
                ),
                "original_model": self.history["model_info"].get(
                    "original_model_path"
                ),
                "model_type": self.history["model_info"].get("model_type"),
                "n_models": self.history["model_info"].get("n_models"),
                "delta": self.history["config"].get("delta"),
                "drift_detector": self.history["model_info"].get(
                    "drift_detector"
                ),
                "drift_detector_params": str(
                    self.history["model_info"].get("drift_detector_params")
                ),
                "top_k": self.history["model_info"].get("top_k"),
                "model_params": str(
                    self.history["model_info"].get("model_params")
                ),
            }

            history_df = pd.DataFrame([history_data])
            base_path = "/content/drive/MyDrive/Pesquisa2024/"
            filename = "training_history.csv"
            filepath = os.path.join(base_path, filename)

            os.makedirs(os.path.dirname(filepath), exist_ok=True)

            if os.path.exists(filepath):
                existing_df = pd.read_csv(filepath)
                history_df = pd.concat(
                    [existing_df, history_df], ignore_index=True
                )

            history_df.to_csv(filepath, index=False)
            print(f"\nHistórico salvo em {filepath}")

        except Exception as e:
            print(f"Erro salvando histórico: {str(e)}")
            traceback.print_exc()

    def train_and_predict_with_sliding_windows(
        self, dataset, window_size=7200, step_size=3600, output_model_path=None
    ):
        """
        Processa o dataset usando janelas deslizantes.

        Parameters:
        -----------
        dataset : pd.DataFrame
            O dataset a ser processado.
        window_size : int
            Tamanho da janela em segundos (default: 1 hora)
        step_size : int
            Tamanho do passo em segundos (default: 30 minutos)
        output_model_path : str, optional
            Caminho para salvar o modelo atualizado.
        """
        try:
            self.validate_dataset(dataset)
            dataset["timestamp"] = pd.to_datetime(
                dataset["click_timestamp"], unit="ms"
            )
            dataset = self.analyze_samples(dataset)

            # Realiza validação temporal primeiro
            val_metrics, val_predictions = self.validate_temporal(
                dataset=dataset.copy(),
                validation_ratio=self.history["config"].get(
                    "validation_ratio", 0.2
                ),
            )

            # Análise das amostras
            print("\nAnalisando amostras...")
            dataset["timestamp"] = pd.to_datetime(
                dataset["click_timestamp"], unit="ms", errors="coerce"
            )
            dataset = self.analyze_samples(dataset)

            # Seleciona os últimos 7 dias mais completos de dados
            daily_counts = dataset.groupby(dataset["timestamp"].dt.date).size()
            active_days = daily_counts[
                daily_counts > 1000
            ].index  # Dias com mais de 1000 interações
            if len(active_days) > 7:
                active_days = sorted(active_days)[-7:]  # Últimos 7 dias ativos
                dataset = dataset[
                    dataset["timestamp"].dt.date.isin(active_days)
                ]
                print(
                    f"\nSelecionados {len(active_days)} dias ativos com {len(dataset):,} registros"
                )

            # Configura informações do dataset
            self.history["dataset_info"] = {
                "total_records": len(dataset),
                "unique_users": dataset["user_id"].nunique(),
                "unique_articles": dataset["click_article_id"].nunique(),
                "start_time": dataset["timestamp"]
                .min()
                .strftime("%Y-%m-%d %H:%M:%S"),
                "end_time": dataset["timestamp"]
                .max()
                .strftime("%Y-%m-%d %H:%M:%S"),
                "window_size": window_size,
                "step_size": step_size,
                "total_days": (
                    dataset["timestamp"].max() - dataset["timestamp"].min()
                ).total_seconds()
                / (24 * 3600),
            }

            # Ordena o dataset pelo timestamp
            dataset = dataset.sort_values("timestamp")

            # Converte timestamps para segundos desde o início
            start_time = dataset["timestamp"].min()
            dataset["seconds"] = (
                dataset["timestamp"] - start_time
            ).dt.total_seconds()

            total_seconds = dataset["seconds"].max()
            window_starts = np.arange(0, total_seconds - window_size, step_size)

            predictions = []
            total_windows = len(window_starts)

            for idx, window_start in enumerate(window_starts):
                window_end = window_start + window_size

                # Seleciona dados da janela atual
                window_mask = (dataset["seconds"] >= window_start) & (
                    dataset["seconds"] < window_end
                )
                window_data = dataset[window_mask]

                print(f"\nProcessando janela {idx + 1}/{total_windows}")
                print(f"Registros na janela: {len(window_data):,}")

                # Processa cada registro na janela
                for _, row in window_data.iterrows():
                    x = row.to_dict()

                    # Treina o modelo
                    self.model.learn_one(x)
                    self.history["processed_records"] += 1

                    # Faz e armazena predição
                    pred = self.model.predict_one(x)
                    target = int(
                        x["click_article_id"] in self.model.current_top
                    )

                    predictions.append(
                        {
                            "timestamp": x["timestamp"],
                            "user_id": x["user_id"],
                            "click_article_id": x["click_article_id"],
                            "prediction": pred,
                            "target": target,
                        }
                    )

                # Report de progresso
                if (idx + 1) % 10 == 0 or idx == len(window_starts) - 1:
                    print(
                        f"\nProgresso: {idx + 1}/{total_windows} janelas processadas"
                    )
                    print(
                        f"Registros processados: {self.history['processed_records']:,}"
                    )
                    print("\nMétricas atuais:")
                    for name, metric in self.model.metrics.items():
                        print(f"{name}: {metric.get():.4f}")

            # Salva todas as predições em um único arquivo
            predictions_df = pd.DataFrame(predictions)
            predictions_filename = f"predictions_{self.timestamp}.csv"
            predictions_path = os.path.join(
                "/content/drive/MyDrive/Pesquisa2024/", predictions_filename
            )
            predictions_df.to_csv(predictions_path, index=False)
            print(f"\nPredições salvas em '{predictions_path}'")

            # Atualiza o histórico com o caminho das predições
            self.history["predictions_file"] = predictions_path

            if output_model_path:
                self.save_model(output_model_path)

            self._update_final_metrics()

            self.history["validation_metrics"] = {
                name: metric.get() for name, metric in val_metrics.items()
            }

            return self.history

        except Exception as e:
            print(f"Erro fatal processando dataset: {str(e)}")
            traceback.print_exc()
            return None

    def save_model_safely(model, path):
        """
        Salva o modelo com verificação
        """
        try:
            # Primeiro salva em um arquivo temporário
            temp_path = path + ".temp"
            with open(temp_path, "wb") as f:
                pickle.dump(model, f)

            # Verifica se pode carregar
            with open(temp_path, "rb") as f:
                _ = pickle.load(f)

            # Se chegou aqui, arquivo está ok
            os.replace(temp_path, path)
            print(f"Modelo salvo com sucesso em {path}")
            return True

        except Exception as e:
            print(f"Erro ao salvar modelo: {str(e)}")
            if os.path.exists(temp_path):
                os.remove(temp_path)
            return False

    '''
    def save_model(self, path):
        """
        Salva o modelo com verificação de segurança

        Parameters:
        -----------
        path : str
            Caminho onde o modelo será salvo
        """
        try:
            # Primeiro salva em um arquivo temporário
            temp_path = path + ".temp"

            # Salva o modelo em arquivo temporário
            with open(temp_path, "wb") as f:
                pickle.dump(self.model, f)

            # Verifica se pode carregar
            with open(temp_path, "rb") as f:
                _ = pickle.load(f)

            # Se chegou aqui, arquivo está ok
            os.replace(temp_path, path)

            # Atualiza informações no histórico
            self.history["model_info"]["final_model_path"] = path
            self.history["model_info"]["save_time"] = datetime.now().strftime(
                "%Y-%m-%d %H:%M:%S"
            )

            print(f"\nModelo salvo com sucesso em {path}")
            return True

        except Exception as e:
            print(f"\nErro ao salvar modelo: {str(e)}")
            if os.path.exists(temp_path):
                os.remove(temp_path)
            return False
            '''

    def save_history(self):
        """
        Salva o histórico do processamento em um arquivo CSV.
        """
        try:
            history_data = {
                "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "dataset_size": self.history["dataset_info"].get(
                    "total_records"
                ),
                "unique_users": self.history["dataset_info"].get(
                    "unique_users"
                ),
                "unique_articles": self.history["dataset_info"].get(
                    "unique_articles"
                ),
                "window_size": self.history["dataset_info"].get("window_size"),
                "step_size": self.history["dataset_info"].get("step_size"),
                "total_days": self.history["dataset_info"].get("total_days"),
                "processed_records": self.history["metrics"].get(
                    "processed_records"
                ),
                "accuracy": self.history["metrics"].get("final_accuracy"),
                "f1_score": self.history["metrics"].get("final_f1"),
                "precision": self.history["metrics"].get("final_precision"),
                "recall": self.history["metrics"].get("final_recall"),
                "roc_auc": self.history["metrics"].get("roc_auc"),
                "log_loss": self.history["metrics"].get("log_loss"),
                "processing_time": self.history["metrics"].get(
                    "processing_time"
                ),
                "n_drifts": self.history["metrics"].get("n_drifts"),
                "n_warnings": self.history["metrics"].get("n_warnings"),
                "top_articles": self.history["metrics"].get("top_articles"),
                "predictions_file": self.history.get("predictions_file"),
                "model_path": self.history["model_info"].get(
                    "final_model_path"
                ),
                "original_model": self.history["model_info"].get(
                    "original_model_path"
                ),
                "model_type": self.history["model_info"].get("model_type"),
                "n_models": self.history["model_info"].get("n_models"),
                "delta": self.history["config"].get("delta"),
                "drift_detector": self.history["model_info"].get(
                    "drift_detector"
                ),
                "drift_detector_params": str(
                    self.history["model_info"].get("drift_detector_params")
                ),
                "top_k": self.history["model_info"].get("top_k"),
                "model_params": str(
                    self.history["model_info"].get("model_params")
                ),
            }

            history_df = pd.DataFrame([history_data])
            base_path = "/content/drive/MyDrive/Pesquisa2024/"
            filename = "training_history.csv"
            filepath = os.path.join(base_path, filename)

            os.makedirs(os.path.dirname(filepath), exist_ok=True)

            if os.path.exists(filepath):
                existing_df = pd.read_csv(filepath)
                history_df = pd.concat(
                    [existing_df, history_df], ignore_index=True
                )

            history_df.to_csv(filepath, index=False)
            print(f"\nHistórico salvo em {filepath}")

        except Exception as e:
            print(f"Erro salvando histórico: {str(e)}")
            traceback.print_exc()


def main(
    input_model: Optional[str] = None,
    dataset_path: Optional[str] = None,
    output_model: Optional[str] = None,
    config: Optional[Dict[str, Any]] = None,
) -> Optional[Dict[str, Any]]:
    """
    Função principal para executar o sistema de recomendação.
    """
    try:
        if not dataset_path:
            raise ValueError("dataset_path é obrigatório")

        # Diretórios
        base_dir = "/content/drive/MyDrive/Pesquisa2024/"
        models_dir = os.path.join(base_dir, "models")
        os.makedirs(models_dir, exist_ok=True)

        # Nome do modelo com timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        if not output_model:
            output_model = os.path.join(
                models_dir, f"news_recommender_{timestamp}.pkl"
            )

        # Configuração padrão
        config = config or {"n_models": 15, "delta": 0.002, "top_k": 10}

        # Inicializa sistema
        system = NewsRecommenderSystem(
            model_path=input_model,
            max_samples=None,
            timestamp=timestamp,
            config=config,
        )

        # Carrega e processa dataset
        print(f"Carregando dataset de {dataset_path}...")
        dataset = pd.read_csv(dataset_path)
        dataset["timestamp"] = pd.to_datetime(
            dataset["click_timestamp"], unit="ms"
        )

        # Processa com janelas deslizantes (1 hora com passo de 30 minutos)
        window_size = 3600  # 1 hora em segundos
        step_size = 1800  # 30 minutos em segundos

        history = system.train_and_predict_with_sliding_windows(
            dataset=dataset,
            window_size=window_size,
            step_size=step_size,
            output_model_path=output_model,
        )

        system.save_history()
        return history

    except Exception as e:
        print(f"Erro na execução principal: {str(e)}")
        traceback.print_exc()
        return None

    # Uso do modelo existente:


def continue_training(
    modelo_anterior_path, novos_dados_path, output_model_path
):
    """
    Continua o treinamento de um modelo existente
    """
    # Carrega modelo anterior
    with open(modelo_anterior_path, "rb") as f:
        modelo = pickle.load(f)

    # Carrega novos dados
    novos_dados = pd.read_csv(novos_dados_path)

    # Inicializa sistema com modelo existente
    system = NewsRecommenderSystem(
        model_path=modelo_anterior_path, config=config
    )

    # Processa novos dados
    history = system.train_and_predict_with_sliding_windows(
        dataset=novos_dados,
        window_size=config["window_size"],
        step_size=config["step_size"],
        output_model_path=output_model_path,
    )

    return history


if __name__ == "__main__":

    # treinamento
    # configurações otimizadas
    config = {
        "n_models": 15,
        "delta": 0.005,
        "top_k": 10,
        "window_size": 7200,  # 2 horas
        "step_size": 3600,  # 1 hora
        "validation_ratio": 0.3,
    }

    history = main(
        dataset_path="/content/drive/MyDrive/Pesquisa2024/dataset_otimizado.csv",
        output_model="/content/drive/MyDrive/Pesquisa2024/models/modelo_recomendacoes1.pkl",
        config=config,
    )

    """
    # Carrega o modelo salvo
    with open('/content/drive/MyDrive/Pesquisa2024/models/news_recommender_new-2.pkl', 'rb') as f:
        modelo = pickle.load(f)

    # Verifica métricas atuais
    print("Métricas do modelo:")
    for nome, metrica in modelo.metrics.items():
        print(f"{nome}: {metrica.get():.4f}")


    # Continua treinamento com novos dados
    history = continue_training(
        modelo_anterior_path='/content/drive/MyDrive/Pesquisa2024/models/news_recommender_new-2.pkl',
        novos_dados_path='/content/drive/MyDrive/Pesquisa2024/dataset_interacoes.csv',
        output_model_path='/content/drive/MyDrive/Pesquisa2024/models/news_recommender_melhorado.pkl'
    )
    """

    if history:
        print("\nProcessamento concluído com sucesso!")
        print("\n=== Estatísticas Finais ===")
        print(
            f"Período total: {history['dataset_info']['total_days']:.1f} dias"
        )
        print(
            f"Total de registros: {history['metrics']['processed_records']:,}"
        )
        print(
            f"Tempo de processamento: {history['metrics']['processing_time']}"
        )

        print("\nMétricas:")
        print(f"Acurácia: {history['metrics']['final_accuracy']:.4f}")
        print(f"F1 Score: {history['metrics']['final_f1']:.4f}")
        print(f"Precisão: {history['metrics']['final_precision']:.4f}")
        print(f"Recall: {history['metrics']['final_recall']:.4f}")
        print(f"ROC AUC: {history['metrics']['roc_auc']:.4f}")
        print(f"Log Loss: {history['metrics']['log_loss']:.4f}")

        print(f"\nDrifts detectados: {history['metrics']['n_drifts']}")
        print(f"Warnings detectados: {history['metrics']['n_warnings']}")

        if history["metrics"].get("top_articles"):
            print("\nTop Artigos:")
            top_articles = eval(history["metrics"]["top_articles"])
            for article, count in top_articles.items():
                print(f"Artigo {article}: {count:,} interações")


Carregando dataset de /content/drive/MyDrive/Pesquisa2024/dataset_otimizado.csv...

=== Análise das Amostras ===
Total de amostras: 2,987,469
Período: 2017-10-01 03:00:00.026000 até 2017-10-17 23:51:27.187000
Usuários únicos: 322,892
Artigos únicos: 45,695

Verificação de integridade:

=== Análise Temporal ===

Distribuição por hora do dia:
hour
0     126548
1     120732
2      94289
3      61809
4      32813
5      18558
6      14518
7      16619
8      32107
9      72778
10    129329
11    184405
12    161904
13    168728
14    182451
15    187951
16    201597
17    191366
18    190417
19    192196
20    179987
21    152274
22    141080
23    133013
dtype: int64

Distribuição por dia:
day
2017-10-01     94056
2017-10-02    303177
2017-10-03    261159
2017-10-04    215415
2017-10-05    190003
2017-10-06    207646
2017-10-07    139323
2017-10-08    108110
2017-10-09    248208
2017-10-10    282391
2017-10-11    238969
2017-10-12    121467
2017-10-13    180723
2017-10-14     95216
2017-1